In [1]:
from bs4 import BeautifulSoup
import requests
# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd

# Random integer for more realistic timing for clicks, buttons and searches during scraping
from random import randint

# Multi Threading
import threading

# Threading:
from concurrent.futures import ThreadPoolExecutor, wait
import math
import mysql.connector
from datetime import date
import json

In [2]:
import selenium

# Check version I am running
selenium.__version__

'4.15.2'

In [3]:
from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [5]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/browser-headers',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
header_browser_list = response.json()
print(header_browser_list['result'][0])

{'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'sec-ch-ua': 'Google Chrome;v="84", "Chromium";v="84", ";Not A Brand";v="99"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': 'macOS', 'sec-fetch-site': 'none', 'sec-fetch-mod': '', 'sec-fetch-user': '?1', 'accept-encoding': 'gzip, deflate', 'accept-language': 'en-US,en;q=0.5'}


In [6]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/user-agents',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
user_agent_list = response.json()
print(user_agent_list['result'][0])

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15


In [7]:
random_index_user_agent = randint(0, len(user_agent_list)-1)
random_index_header_browser = randint(0, len(header_browser_list)-1)
user_agent_random = user_agent_list['result'][random_index_user_agent]
header_browser_random = header_browser_list['result'][random_index_header_browser]

In [8]:
# Allows you to cusotmize: ingonito mode, maximize window size, headless browser, disable certain features, etc
option = webdriver.ChromeOptions()

# Going undercover:
option.add_argument("--incognito")


# # Consider this if the application works and you know how it works for speed ups and rendering!

option.add_argument('--headless=chrome')
user_agent = user_agent_random
option.add_argument(f"user-agent={user_agent}")

# Thêm header vào Options
headers = header_browser_random

for key, value in headers.items():
    option.add_argument(f"--header={key}:{value}")

In [9]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=option)
driver.implicitly_wait(10)  # Set the implicit wait time to 10 seconds
driver.set_page_load_timeout(30)
driver.get("https://www.topcv.vn/viec-lam-it")

job_count = driver.find_element(By.CSS_SELECTOR,'[class="job-header"] h2 b').text.replace(",", "")
if (int(job_count) % 50 == 0):
    max_page  = int(job_count) / 50
else:
    max_page = math.floor(int(job_count) / 50) + 1
print(max_page)
driver.quit()

46


In [10]:
job_urls = []
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=option)
for page_number in range(1, max_page+1):
    driver.get(f"https://www.topcv.vn/viec-lam-it?page={page_number}")
    job_url_list = driver.find_elements(By.CSS_SELECTOR, '.job-item-2 h3[class="title"] a')
    for uu in job_url_list:
        link = uu.get_attribute("href")
        job_urls.append(link)
driver.quit()

In [27]:
conn = mysql.connector.connect(
    host='103.56.158.31',
    port = 3306,
    user= 'tuyendungUser',
    password='sinhvienBK',
    database= 'ThongTinTuyenDung'
)
cursor = conn.cursor()

In [28]:
sql = 'INSERT IGNORE INTO Stg_ThongTin(Web, Nganh, Link, TenCV, CongTy, TinhThanh, Luong, LoaiHinh, KinhNghiem, CapBac, HanNopCV, YeuCau, MoTa, PhucLoi, SoLuong) VALUES (%s, %s, %s, %s,%s,%s,%s, %s, %s, %s, %s, %s, %s, %s,%s)'

In [32]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=option)
for job_url in job_urls:
    if "https://www.topcv.vn/brand/" in job_url:
        continue
    link = job_url
    print("Đang xử lí....", link)
    driver.get(link)
    Web = 'TopCV'
    Nganh = 'IT'
    Link = link
    TenCV = driver.find_element(By.CSS_SELECTOR, '.job-detail__info--title ').text
    CongTy = driver.find_element(By.CSS_SELECTOR, '[class="company-name-label"] a').text
    SoLuong = "1"
    YeuCau = ""
    MoTa = ""
    PhucLoi = ""
    for i in range(len(driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]'))):
        if "Mức lương" in driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-title"]').text:
            Luong = driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-value"]').text
        if "Địa điểm" in driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-title"]').text:
            TinhThanh = driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-value"]').text
    try:
        HanNopCV = driver.find_element(By.CSS_SELECTOR, '[class="job-detail__info--deadline"]').text.split(":")[-1].strip()
    except:
        HanNopCV = date.today()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    List = soup.find_all('div', class_='job-description__item')
    YeuCau = List[1].text
    MoTa = List[0].text
    PhucLoi = List[2].text
    for i in range(len(soup.find_all('div', class_='box-general-group'))):
        if "Số lượng tuyển" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
            SoLuong = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text.split()[0]
        if "Hình thức làm việc" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
            LoaiHinh = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text
        try:
            if "Kinh nghiệm" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
                KinhNghiem = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text
        except:
            KinhNghiem = "Không có"
        if "Cấp bậc" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
            CapBac = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text
    cursor.execute(sql, (Web, Nganh, Link, TenCV, CongTy, TinhThanh, Luong, LoaiHinh, KinhNghiem, CapBac, HanNopCV, YeuCau, MoTa, PhucLoi, SoLuong))
    conn.commit()
driver.quit()
cursor.close()
conn.close()


Đang xử lí.... https://www.topcv.vn/viec-lam/lap-trinh-vien-java-du-an-banking-upto-30m-net-luong-thang-13-upto-100/190591.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/fresher-java-tieng-nhat-dao-tao-tieng-nhat-thu-nhap-10-15m/1188454.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/chuyen-vien-it-it-helpdesk/1192008.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/java-developer-tai-ha-noi-luong-tu-15-40-trieu/397896.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/chuyen-vien-phat-trien-san-pham-product-owner-rd-luong-upto-50tr-truyen-hinh-mytv-tai-ha-noi/1182255.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/chuyen-vien-cong-nghe-thong-tin-tai-tp-vinh/1155702.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/front-end-developers-thu-nhap-upto-2-600usd/1197636.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... 

In [15]:
print(job_urls[0])

https://www.topcv.vn/viec-lam/lap-trinh-vien-java-du-an-banking-upto-30m-net-luong-thang-13-upto-100/190591.html?ta_source=ITJobs_LinkDetail
